<a href="https://colab.research.google.com/github/parkmina365/machine_learning_guide/blob/main/CH2_%EC%82%AC%EC%9D%B4%ED%82%B7%EB%9F%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. sklearn
- 모형 예측. 머신러닝의 가장 대표적인 라이브러리
- 머신러닝 분석. 결과 관점에서 접근. 모형예측 평가에 특화
- 6가지 기능
   - 지도학습: 분류(Classification), 회귀(Regression)
   - 비지도학습 : 범주화(Clustering), 차원축소/ 피처 추출(Feature extraction)
   - 기타 : 모델선택 및 평가, 데이터 전처리 
- 지도학습 : 명확한 정답(Label)이 주어진 데이터(Feature)를 학습 데이터 세트(Train)로 학습한 뒤 미지의 정답을 테스트 데이터 세트(Test)로 예측하는 방식
- 예) LinearRegression, preprocessing, train_test_split, linear_model, metrics, Encoding


\- statsmodels 라이브러리와의 비교
   - 통계 분석 및 추정. 모델 형성. 머신러닝도 가능하나 통계모델에 특화되어 있음
   - 예) smf.ols, sm.qqplot, sm.stats.anova_lm, qq-plot, ANOVA

# 2. 예시: 붓꽃 품종 예측
- 분류(Classification)기법 : 대표적 지도학습 방법
- DecisionTreeClassifier 사용
- load_iris : Feature(독립변수) 4가지를 기준으로 Label(종속변수) 3가지로 분류 하는 데이터

In [ ]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd

In [ ]:
# df 만들기
iris = load_iris()
print(iris.keys())
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_df['label'] = iris.target
iris_df.head()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename'])


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),label
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [ ]:
# train, test 분리하기
X = iris_df.iloc[:,:-1]
y = iris_df.iloc[:,-1]
X_train, X_test, y_train, y_test = \
    train_test_split(X,y,test_size=0.2,random_state=1)

In [ ]:
# DecisionTreeClassifier 객체로 학습, 예측, 평가
dt_clf = DecisionTreeClassifier()  # 객체 생성
dt_clf.fit(X_train, y_train)       # 학습
pred = dt_clf.predict(X_test)      # 예측
accuracy_score(y_test, pred)       # 평가

0.9666666666666667

# 3. sklearn 프레임워크
- 지도학습 클래스(Estimator) : Classifier, Regressor 등 / fit(), predict()
- 비지도학습 클래스 : fit(), transform(), fit_transform()
- 평가 클래스 : cross_val_score
- 하이퍼파라미터 튜닝 지원 클래스 : GridSearchCV
- sklearn 주요 모듈
   - sklearn.datasets
   - sklearn.preprocessing: 전처리(인코딩, 정규화, 스케일링)
   - sklearn.model_selection: 교차검증을 위한 학습/테스트 데이터 분리. GridSearch로 최적파라미터 추출 등
   - sklearn.metrics: 성능 측정(분류, 회귀, 클러스터링 등). Accuracy, Precision, Recall, ROC-AUC, RMSE 등
   - 알고리즘
     - sklearn.ensenble: 앙상블 알고리즘. RandomForest, GBM 등
     - sklearn.linear_model: linear, Logistic, Ridge, Lasso 등 회귀 알고리즘
     - sklearn.neighbors: 최근접 이웃 알고리즘인 KNN 등
     - sklearn.svm: Support Vector Machine
     - sklearn.tree: 의사결정트리
     - sklearn.cluster: 클러스터링 알고리즘. K-평균, 계층형 등
     - sklearn.naive_bayes: 나이브 베이즈 알고리즘. 가우시안 NB 등

# 4. sklearn.model_selection
- 교차검증을 위한 학습/테스트 데이터 분리. 하이퍼파라미터 튜닝 등
- 예) train_test_split, GridSearchCV 

## 교차검증(CV:Cross Validation)
- 학습용 데이터의 편중을 막기 위해 별도로 여러 세트로 구성된 학습용/테스트용 데이터 세트에서 학습과 평가를 여러번 수행. 각 세트에서 수행한 평가 결과에 따라 하이퍼 파라미터 튜닝 등의 모델 최적화가 가능
- 학습용 데이터와 테스트용 데이터를 분리해서 모델링 및 평가를 수행하는 방식은 과적합에 취약한 약점이 있음. 고정된 테스트 데이터로만 평가를 하게되면 테스트 데이터에만 최적의 성능을 발휘할 수 있도록 편향되게 모델을 유도하는 경향이 발생. 문제점을 개선하기 위하여 교차 검증(Cross Validation)을 이용해 다양한 학습과 평가를 수행
  - 과적합: 모델이 학습 데이터에만 과도하게 최적화되어, 실제 예측을 다른 데이터로 수행할 때 예측 성능이 과도하게 떨어지는 것

### 1. KFold
- K개의 데이터 폴드 세트를 만들어 K번만큼 각 폴드 세트에 학습과 검증 평가를 반복적으로 수행. 전체 데이터를 K등분한 인덱스를 뽑아주는 개념
- 데이터 세트를 K등분, 학습용 데이터와 테스트용 데이터를 K번 변경하면서 학습과 검증을 수행한 결과를 평균해서 평가 결과 산출
- K번의 평가 점수의 평균으로 예측 성능을 평가함

- https://m.blog.naver.com/ckdgus1433/221599517834

In [ ]:
# KFold로 교차검증
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import numpy as np

features = iris.data
label = iris.target
dt_clf = DecisionTreeClassifier()

# K=5 개의 Fold로 분리하는 KFold 객체 생성
kfold = KFold(n_splits=5)

# Fold 세트별 평가(정확도)할 리스트 생성
cv_accuracy =[]

# KFold의 split(): Fold별 학습용, 테스트용 데이터의 index를 array로 반환
n_iter = 0
for train_index, test_index in kfold.split(features):
    # 학습용, 테스트용 데이터 분리
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    
    # 학습, 예측
    dt_clf.fit(X_train,y_train)
    pred = dt_clf.predict(X_test)
    n_iter += 1

    # 매 K마다 평가(정확도)
    accuracy = accuracy_score(y_test,pred)
    cv_accuracy.append(accuracy)
    print(f'#{n_iter} CV 정확도 : {accuracy}\nTest index : \n{test_index}\n')

print(f'평균 CV 정확도 : {np.mean(cv_accuracy):.4f}')

### Test index가 랜덤하지 않게 고르게 나뉘어져 있음

#1 CV 정확도 : 1.0
Test index : 
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]

#2 CV 정확도 : 1.0
Test index : 
[30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
 54 55 56 57 58 59]

#3 CV 정확도 : 0.8333333333333334
Test index : 
[60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83
 84 85 86 87 88 89]

#4 CV 정확도 : 0.9333333333333333
Test index : 
[ 90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]

#5 CV 정확도 : 0.8333333333333334
Test index : 
[120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]

평균 CV 정확도 : 0.9200


### 2. Stratified KFold
- 원본 데이터의 레이블 분포를 먼저 고려한 뒤 이 분포와 동일하게 학습용, 테스트용 데이터세트를 분배함
- KFold가 레이블 데이터 집합이 원본 데이터 집합의 레이블 분포를 학습 및 테스트 세트에 제대로 분배하지 못하는 문제점을 개선함
- 불균형한 분포도를 가진 레이블 데이터 집합을 위한 KFold 방식
- Classfication의 CV는 Stratified KFold로 분할되어야 함
- Regression의 label은 이산이 아닌 연속이기에 Stratified KFold 사용불가

In [ ]:
# KFold의 문제점
kfold = KFold(n_splits=3)
n_iter=0
for train_index, test_index in kfold.split(iris_df):
    n_iter +=1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    print(f'Fold:{n_iter}')
    print(f'Train label value counts : \n{label_train.value_counts()}')
    print(f'Test label value counts : \n{label_test.value_counts()}\n')

### Fold마다 Train과 Test의 label 분포가 다름

Fold:1
Train label value counts : 
1    50
2    50
Name: label, dtype: int64
Test label value counts : 
0    50
Name: label, dtype: int64

Fold:2
Train label value counts : 
0    50
2    50
Name: label, dtype: int64
Test label value counts : 
1    50
Name: label, dtype: int64

Fold:3
Train label value counts : 
0    50
1    50
Name: label, dtype: int64
Test label value counts : 
2    50
Name: label, dtype: int64



In [ ]:
# Stratified KFold 사용시 train, test label의 분포
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=3)
n_iter=0

for train_index, test_index in skf.split(iris_df, iris_df['label']): # label 추가
    n_iter +=1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    print(f'Fold:{n_iter}')
    print(f'Train label value counts : \n{label_train.value_counts()}')
    print(f'Test label value counts : \n{label_test.value_counts()}\n')

Fold:1
Train label value counts : 
2    34
0    33
1    33
Name: label, dtype: int64
Test label value counts : 
0    17
1    17
2    16
Name: label, dtype: int64

Fold:2
Train label value counts : 
1    34
0    33
2    33
Name: label, dtype: int64
Test label value counts : 
0    17
2    17
1    16
Name: label, dtype: int64

Fold:3
Train label value counts : 
0    34
1    33
2    33
Name: label, dtype: int64
Test label value counts : 
1    17
2    17
0    16
Name: label, dtype: int64



In [ ]:
# StratifiedKFold로 교차검증(CV)

dt_clf = DecisionTreeClassifier()
skfold = StratifiedKFold(n_splits=3)
n_iter = 0
cv_accuracy = []

for train_index, test_index in skfold.split(features, label):
    n_iter +=1
    features_train = features[train_index]
    features_test = features[test_index]
    label_train = label[train_index]
    label_test = label[test_index]

    dt_clf.fit(features_train, label_train) # 학습
    pred = dt_clf.predict(features_test)    # 예측
    accuracy = accuracy_score(label_test, pred) # 평가
    cv_accuracy.append(accuracy)
    print(f'Fold:{n_iter}\n정확도 : {accuracy_score(label_test, pred)}')
    print(f'Test index : \n{test_index}\n')
print(f'평균 정확도 : {np.mean(cv_accuracy):.4f}')

Fold:1
정확도 : 0.98
Test index : 
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  50
  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115]

Fold:2
정확도 : 0.94
Test index : 
[ 17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  67
  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82 116 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132]

Fold:3
정확도 : 0.96
Test index : 
[ 34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  83  84
  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 133 134 135
 136 137 138 139 140 141 142 143 144 145 146 147 148 149]

평균 정확도 : 0.9600


### 3. cross_val_score
- Stratified KFold(회귀는 KFold)의 일련의 과정(학습,예측,평가)을 한꺼번에 수행 후 평가 지표를 출력
- cross_val_score()는 하나의 평가 지표, cross_validate()는 여러개의 평가 지표
- 주요 파라미터
  - estimator: 모델 객체
  - X, y
  - scoring : 평가방식(accuracy 등)
  - cv : fold 수


In [ ]:
from sklearn.model_selection import cross_val_score, cross_validate

dt_clf = DecisionTreeClassifier()
score = cross_val_score(dt_clf, features, label, scoring='accuracy', cv=3)
print(f'CV별 정확도 : {score}\n평균 정확도 : {np.mean(score)}')

CV별 정확도 : [0.98 0.94 0.96]
평균 정확도 : 0.96


### 4. GridSearchCV
- CV 기반, 알고리즘에 사용되는 파라미터를 순차적으로 입력하며 예측 성능을 개선하는 최적의 파라미터(하이퍼 파라미터)을 도출함
- Grid: 격자. 촘촘하게 여러 파라미터들을 엮어 테스트 함
- CV(교차검증)와 하이퍼파라미터 튜닝을 한 번에 함
- 주요 파라미터 : estimator, param_grid, scoring, cv, refit
  - param_grid : estimator 튜닝을 위해 딕셔너리 형식으로 파라미터명과 사용될 파라미터들의 값을 지정
  - refit : 디폴트 True. True로 생성 시 가장 최적의 하이퍼 파라미터를 찾은 뒤, 입력된 estimator 객체를 해당 하이퍼 파라미터로 재학습시킨다

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import pandas as pd 

# train, test 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(features, label,\
     test_size=0.2, random_state=0)
dt_clf = DecisionTreeClassifier()

# GridSearchCV 객체 만들기
params = {'max_depth':[1,2,3],'min_samples_split':[2,3]}
grid_dtree = GridSearchCV(estimator=dt_clf, param_grid=params, cv=3)

# 학습
grid_dtree.fit(X_train,y_train)

# 결과 출력
df = pd.DataFrame(grid_dtree.cv_results_)
display(df[['params','mean_test_score','rank_test_score','split0_test_score','split1_test_score','split2_test_score']])
print(grid_dtree.cv_results_.keys())
print(f'\n최적 하이퍼 파라미터 : {grid_dtree.best_params_}')
print(f'최고 예측 정확도 : {grid_dtree.best_score_:.4f}')

# 최적 파라미터 이용한 정확도 측정
estimator = grid_dtree.best_estimator_
pred = estimator.predict(X_test)
print(f'테스트 데이터 세트 정확도 : {accuracy_score(y_test,pred):.4f}')

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.691667,5,0.700,0.700,0.675
1,"{'max_depth': 1, 'min_samples_split': 3}",0.691667,5,0.700,0.700,0.675
2,"{'max_depth': 2, 'min_samples_split': 2}",0.925000,3,0.850,0.975,0.950
3,"{'max_depth': 2, 'min_samples_split': 3}",0.925000,3,0.850,0.975,0.950
4,"{'max_depth': 3, 'min_samples_split': 2}",0.925000,1,0.925,0.900,0.950
5,"{'max_depth': 3, 'min_samples_split': 3}",0.925000,1,0.925,0.900,0.950


dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_max_depth', 'param_min_samples_split', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])

최적 하이퍼 파라미터 : {'max_depth': 3, 'min_samples_split': 2}
최고 예측 정확도 : 0.9250
테스트 데이터 세트 정확도 : 0.9667


# 5. sklearn.preprocessing
- feature 전처리 과정: Encoding, Scaling 등
- 머신러닝 알고리즘은 문자열 값을 입력 값으로 허용하지 않음. 인코딩하여 숫자 값으로 변환해야
  - 카테고리형 feature: 코드값으로 표현
  - 텍스트형 feature: feature vectorization으로 벡터화

## 1. LabelEncoder
- 카테고리형 feature를 코드형 숫자값으로 변환
- 숫자값은 단순 코드이지 숫자값에 따른 순서나 중요도로 인식되면 안됨. 선형회귀 등에는 적용하면 안됨

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Encoding
items = ['TV', '냉장고', '전자렌지', '컴퓨터', '선풍기', '선풍기', '믹서', '믹서']
encoder = LabelEncoder()
labels = encoder.fit_transform(items)
print(f'인코딩 변환값 : {labels}')
print(f'인코딩 클래스 : {encoder.classes_}')

# Decoding
print(f'디코딩 원본값 : {encoder.inverse_transform(labels)}')

인코딩 변환값 : [0 1 4 5 3 3 2 2]
인코딩 클래스 : ['TV' '냉장고' '믹서' '선풍기' '전자렌지' '컴퓨터']
디코딩 원본값 : ['TV' '냉장고' '전자렌지' '컴퓨터' '선풍기' '선풍기' '믹서' '믹서']


## 2. OneHotEncoder
- LabelEncoding의 단점 보완. feature값의 유형에 따라 새로운 feature를 추가해 고유값에 해당하는 칼럼에만 1표시하고 나머지엔 0표시
- One-Hot: 여러개의 속성 중 단 한 개의 속성만 1로 표시
- 문자형 데이터가 아닌 숫자형 데이터로 변환 후에 사용 가능하며, 2차원 데이터형태로 입력해야함

In [ ]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

# LabelEncoding 먼저 실시
items = ['TV', '냉장고', '전자렌지', '컴퓨터', '선풍기', '선풍기', '믹서', '믹서']
encoder_l = LabelEncoder()
labels = encoder_l.fit_transform(items).reshape(-1,1)

# OneHotEncoding
encoder = OneHotEncoder()
labels1 = encoder.fit_transform(labels)
labels1.toarray()

array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.]])

In [ ]:
# pd.get_dummies 이용
import pandas as pd
pd.get_dummies(items)

,TV,냉장고,믹서,선풍기,전자렌지,컴퓨터
0,1,0,0,0,0,0
1,0,1,0,0,0,0
2,0,0,0,0,1,0
3,0,0,0,0,0,1
4,0,0,0,1,0,0
5,0,0,0,1,0,0
6,0,0,1,0,0,0
7,0,0,1,0,0,0


## 3. StandardScaler
- Feature Scaler 중 하나
- 표준정규분포로 표준화(Standardization)
- RBF 커널을 이용하는 SVM, LinearRegression, Logistic Regression은 데이터를 정규분포로 가정하고 구현되었기에 feature를 표준화하는게 좋음

In [ ]:
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
import pandas as pd

iris = load_iris()
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)
print(iris_df.mean())
print()

# StanardScaling
scaler = StandardScaler()
iris_scaled = scaler.fit_transform(iris_df)  # ndarray
iris_scaled_df = pd.DataFrame(iris_scaled, columns=iris.feature_names)
print(iris_scaled_df.mean().round(2))

sepal length (cm)    5.843333
sepal width (cm)     3.057333
petal length (cm)    3.758000
petal width (cm)     1.199333
dtype: float64

sepal length (cm)   -0.0
sepal width (cm)    -0.0
petal length (cm)   -0.0
petal width (cm)    -0.0
dtype: float64


## 4. MinMaxScaler
- 0과 1사이의 값으로 정규화(Normalization)
- 음수 값이 있을시, -1에서 1사이 값으로 변환
- 데이터가 정규분포가 아닐시 시행


In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
iris_scaled = scaler.fit_transform(iris_df)
iris_scaled_df = pd.DataFrame(iris_scaled, columns=iris.feature_names)
display(iris_scaled_df.mean(), iris_scaled_df.describe())

sepal length (cm)    0.428704
sepal width (cm)     0.440556
petal length (cm)    0.467458
petal width (cm)     0.458056
dtype: float64

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
count,150.000000,150.000000,150.000000,150.000000
mean,0.428704,0.440556,0.467458,0.458056
std,0.230018,0.181611,0.299203,0.317599
min,0.000000,0.000000,0.000000,0.000000
25%,0.222222,0.333333,0.101695,0.083333
50%,0.416667,0.416667,0.567797,0.500000
75%,0.583333,0.541667,0.694915,0.708333
max,1.000000,1.000000,1.000000,1.000000
